In [26]:
# Test block, only used for quick testing.
# Note that defined variables is inherited in other code blocks.
import numpy as np

a = np.array([3,1,1,1])
b = np.ones(3)
c = np.column_stack((np.ones(4), a))

c.shape


(4, 2)

In [ ]:
from pathlib import Path
import numpy as np
from numpy.typing import NDArray

def test():
    project_root = Path(__file__).resolve().parents[3]
    relative_path = Path(__file__).resolve().relative_to(project_root)
    print(f"Hello from `{relative_path}` <3")

def sigmoid_function(z: NDArray[np.float64]) -> NDArray[np.float64]:
    return np.exp(z) / (1 + np.exp(z))



def generat_logreg_data(
        n: int,
        beta: NDArray[np.float64],
        seed: int = 0,
) -> tuple[NDArray[np.float64], NDArray[np.int64]]:
    
    #1 Compute the number of covariates from the length of the beta vector
    beta_len = beta.shape[0] - 1 # has to be -1, else the intercept is included (generated with matrix_ones)
    
    #2 Create a random number generator called 'rng'
    rng = np.random.default_rng()
    
    #3 Simulate a matrix X with shape (N, p+1).
        #3.1 First column is all 1.
    matrix_ones = np.ones(n)
        #3.2 The others are drawn from a uniform dist ~Unif(-1,1)
    matrix_X_draw_uniform = rng.integers(low=-1, high=2, size=(n,beta_len))

    X = np.column_stack((matrix_ones, matrix_X_draw_uniform))
        
    #4 Draw y by using rng.binomial with n=1 and prob param = p (not num of covariates)
    z = X @ beta
    p = sigmoid_function(z)
    y = rng.binomial(1, p)

    return X, y


def irls(
    X: NDArray[np.float64],
    y: NDArray[np.int64],
    tol: float = 1e-8, # Tolerance. See stop condition
    max_iter: int = 100,
):
    """Iteratively reweighted least squares solver."""
    n, p = X.shape # amount of draws and covariates.
    converged = False # convergence flag
    y = y.reshape(-1,1).astype(np.float64) # Make 'y' a column vector
    beta = np.ones([p, 1]) # see comment about initialising beta as X.shape[0]
    #print(beta)

    
    for t in range(0,max_iter):

        # compute probability
        p_hat = sigmoid_function(X @ beta)

        # Weights from the diagonal
        W = (p_hat * (1-p_hat))

        # Calculate Z
        Z = X @ beta + (y - p_hat) / W

        # Apply weights
        WX = X * W
        # Do the New 
        beta_next = np.linalg.inv(X.T @ WX) @ (X.T @ (W * Z))

        # Stop condition. 
        if np.any(np.abs(beta_next - beta) < tol):
            """
            Why is the original "(beta_next - beta) > tol", when we clearly are checking if its below the tolerance?
            Else we have to include a not in the "if" statement -> its confusing
            """
            converged = True
            beta = beta_next
            break

        beta = beta_next
        
    return {
        "estimates": beta.flatten(),
        "iterations": t,
        "converged": converged
    }
    
    
    # Check whether the algorithm converged each iteration
        # Max iterations is set to 100
    # Return a dictionary with 'estimates, iterations and converged'
        # Estimates as a 1d numpy array shape (p+1)
        # Iterations = the number of iterations until converd
        # Converged = True flag if <= 100, else false
    """ # Initialize beta to (X.shape[0],1) column vector.
    What? X.shape[0], 1 is a (n,1) -> X.shape gives a (n,p)
    Shouldn't it be (p,1) column vector? Works if it's p, also makes sense since we feed beta in generat_logreg_data with a 3x1 array.
    Also, what
    """





X, y = generat_logreg_data(10000, np.array([1,1,-1]), seed = 1999)
irls(X,y)



{'estimates': array([ 1.04488458,  1.02109938, -1.04712233]),
 'iterations': 5,
 'converged': True}